# Analysis of forming test data

This analysis starts with data exported from the GOM-correlate software.

Each experiment has 3 sections and each section has data for the major and minor strain. Each experiment is repeated multiple times with multiple sample geometries.

First the data are extracted the full strain path from initial stage to the last stage before crack formation and till the onset of  necking, respectively.

In [ ]:
from pathlib import Path
import math

import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

experiment_directory = "../Data/Strain data_All stages/"
experiment_paths = list(Path(experiment_directory).glob("*"))
print(f"{len(experiment_paths)} experiments found in experiment directory.")
experiment_names = [path.name for path in experiment_paths]

In [ ]:
# Loop through all of the frames and find the last one before the sample breaks

for path in tqdm(experiment_paths):
    major_strain_frames = []
    data_directory = path / Path("section one/major strain/")
    file_list = list(data_directory.glob("*.csv"))
    for file in file_list:
        data = np.genfromtxt(file, skip_header=6, delimiter=";", usecols=[5])
        if np.isfinite(data[0]):
            major_strain_frames.append(data)
        else:
            break_time = len(major_strain_frames) + 1
            break

    # Find the position of the maximum strain as it breaks
    max_strain_index = np.argmax(major_strain_frames[-1])

    # Now collect strain over time at the breaking point
    major_strain = [timestep[max_strain_index] for timestep in major_strain_frames]

    # Collect the same data but for minor strains
    minor_strain = []

    for file_index in range(1, break_time):
        file_path = path / Path(f"section one/minor strain/Section1_{file_index:04d}.csv")
        data = np.genfromtxt(file_path, skip_header=6, delimiter=";", usecols=[5])
        minor_strain.append(data[max_strain_index])

    plt.plot(minor_strain, major_strain, "-x")
plt.xlabel("minor strain")
plt.ylabel("major strain")
plt.title(f"Strain paths for Surflex")
plt.savefig("../Results/strain_path_to_break.png", dpi=200)